In [3]:
import json
import logging
import os
import sys

from dotenv import load_dotenv
from loguru import logger
import numpy as np
import pandas as pd
import s3fs
import sqlalchemy
import boto3
from sqlalchemy import text

import geopandas as gpd
from shapely.geometry import LineString

sys.path.append('..')

from src.preprocessing import get_country_name_general

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

project_path = 's3://sfgdata/projects/sustainable-transport'
dataset_name = 'oerebro_nshift_2022'


## EDA

In [67]:
df = pd.read_csv(f'{project_path}/raw/Shipment Report RDC Örebro 2022-01-01 - 2022-12-31.csv', delimiter = ';', decimal = '.')

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_33560\1999910084.py:1: DtypeWarning: Columns (8,9,14,15,17,23,26,28,31,32,37,47,48,53,55,57,62,67,69,76,91,99,100,101,102,107,108,110) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{project_path}/raw/Shipment Report RDC Örebro 2022-01-01 - 2022-12-31.csv', delimiter = ';', decimal = '.')


In [4]:
df = pd.read_csv(f'{project_path}/processed/poc2.csv')

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_37416\717032677.py:1: DtypeWarning: Columns (1,5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{project_path}/processed/poc2.csv')


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538719 entries, 0 to 538718
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Pickup date         538719 non-null  object 
 1   Client no           537874 non-null  object 
 2   DC name             538719 non-null  object 
 3   Client name         538719 non-null  object 
 4   Address             538601 non-null  object 
 5   Zip                 538700 non-null  object 
 6   City                506409 non-null  object 
 7   Country code        538719 non-null  object 
 8   Carrier name        538719 non-null  object 
 9   Sender weight (kg)  538719 non-null  float64
 10  Volume (m3)         538719 non-null  float64
 11  Nb of packages      131328 non-null  float64
 12  DC code             538719 non-null  object 
 13  Shipping type       538719 non-null  object 
 14  Country             538719 non-null  object 
 15  DC Name             538719 non-nul

In [5]:
df.dropna(subset = ['Distance (km)'])

,Pickup date,Client no,DC name,Client name,Address,Zip,City,Country code,Carrier name,Sender weight (kg),Volume (m3),Nb of packages,DC code,Shipping type,Country,DC Name,DC address,DC city,DC zip,DC country,DC country code,Shipper longitude,Shipper latitude,Receiver longitude,Receiver latitude,Distance (km),Co2 emissions,Co2 diesel,Co2 BEV
0,2022-01-02 00:00:00,37580258-230,RDC Örebro,AHLSELL AB / GELIA,INDUSTRIVÄGEN,46784,GRÄSTORP,SE,DHL Sverige (SE),73.000,611.9490,1.0,DK01,Road,Sweden,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,12.080903,57.573191,295.740,2158.902000,1791.888660,40.155577
1,2022-01-02 00:00:00,658053,RDC Örebro,VARD ELECTRO HASTELEVERING,TENNFJORDVEGEN 113,6264,TENNFJORD,NO,Bring (NO),21.461,35.8189,8.0,DK01,Road,Norway,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,6.589492,62.533842,754.105,1618.384740,1343.259335,30.101956
2,2022-01-02 00:00:00,656779,RDC Örebro,VARD ELECTRO HASTELEVERING,TENNFJORDVEGEN 113,6264,TENNFJORD,NO,Bring (NO),161.000,365.6484,1.0,DK01,Road,Norway,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,6.589492,62.533842,754.105,12141.090500,10077.105115,225.824283
3,2022-01-02 00:00:00,181121EEV,RDC Örebro,OUKOTA OY,Valikkokuja 1,90410,NaN,FI,Posti Oy (FI),0.114,0.2025,1.0,DK01,Road,Finland,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,25.511509,64.952434,1240.796,14.145074,11.740412,0.263098
4,2022-01-02 00:00:00,4503157646,RDC Örebro,Konecranes Finland Oy,Läntinen Yhdystie 210/Ovi U5,5830,NaN,FI,Posti Oy (FI),1.215,3.7978,2.0,DK01,Road,Finland,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,24.831594,60.626845,683.255,83.015483,68.902850,1.544088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491859,2022-12-30 18:02:00,1500077276,DC Evreux,SONEPAR FR DIST ST VULB ST 109,345 ALLEE DE CUREBOURSE,1150,SAINT VULBAS,FR,TRANSPORTS BREGER,16557.854,118.3490,NaN,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,5.282961,45.855417,577.005,955396.454727,792979.057423,76431.716378
491860,2022-12-30 18:07:00,1000065092,DC Evreux,EFA FRANCE SARL,3 RUE CLAUDE CHAPPE,37230,FONDETTES,FR,TRANSPORTS BREGER,3342.139,14.6210,NaN,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,0.624121,47.411070,270.988,90567.956333,75171.403757,7245.436507
491861,2022-12-30 18:07:00,1500010296,DC Evreux,STAC CDL STOCK,300 AV ANTOINE LAVOISIER,44590,DERVAL,FR,TRANSPORTS BREGER,1357.641,10.8360,NaN,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-1.662835,47.644423,369.566,50173.795381,41644.250166,4013.903630
491862,2022-12-30 18:07:00,1000071197,DC Evreux,SOREEL SAS,18 RUE DE LA GATINE,49300,CHOLET,FR,TRANSPORTS BREGER,1040.999,7.3940,NaN,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-0.920033,47.037756,336.152,34993.389585,29044.513355,2799.471167


In [78]:
dataset_name = 'sql_poc2'
df1 = pd.read_csv(f'{project_path}/processed/{dataset_name}.csv', dtype={'Zip': str})
df2 = pd.read_csv(f'{project_path}/processed/{dataset_name}.csv', dtype={'Zip': str})
    
# Dict for column names
pd.concat([df1, df2], axis = 0)

,Pickup date,DC code,DC name,Client no,Client name,Address,Zip,City,Country code,Shipping type,Carrier name,Sender weight (kg),Volume (m3),Nb of packages,Country,DC Name,DC address,DC city,DC zip,DC country,DC country code,Shipper longitude,Shipper latitude,Receiver longitude,Receiver latitude,Distance (km),Co2 emissions
0,2022-01-03 08:18:00,FR43,DC Newlog,1000051992,SOULES SAS,45 CHEMIN D'OURS,65420,IBOS,FR,41,DPD FRANCE,1.815,0.037,NaN,France,DC Newlog,22 rue des Garinnes,St Quentin Fallavier,38070,France,FR,5.122075,45.652620,0.010263,43.240029,707.002,128.320863
1,2022-01-19 11:20:00,FR43,DC Newlog,1000051992,SOULES SAS,45 CHEMIN D'OURS,65420,IBOS,FR,49,SCHENKER FRANCE,53.774,0.648,NaN,France,DC Newlog,22 rue des Garinnes,St Quentin Fallavier,38070,France,FR,5.122075,45.652620,0.010263,43.240029,707.002,3801.832555
2,2022-02-16 16:57:00,FR43,DC Newlog,1000051992,SOULES SAS,45 CHEMIN D'OURS,65420,IBOS,FR,41,DPD FRANCE,1.626,0.021,NaN,France,DC Newlog,22 rue des Garinnes,St Quentin Fallavier,38070,France,FR,5.122075,45.652620,0.010263,43.240029,707.002,114.958525
3,2022-01-11 10:29:00,FR43,DC Newlog,1000051992,SOULES SAS,45 CHEMIN D'OURS,65420,IBOS,FR,49,SCHENKER FRANCE,55.526,0.751,NaN,France,DC Newlog,22 rue des Garinnes,St Quentin Fallavier,38070,France,FR,5.122075,45.652620,0.010263,43.240029,707.002,3925.699305
4,2022-01-07 10:32:00,FR43,DC Newlog,1000051992,SOULES SAS,45 CHEMIN D'OURS,65420,IBOS,FR,49,SCHENKER FRANCE,9.176,0.504,NaN,France,DC Newlog,22 rue des Garinnes,St Quentin Fallavier,38070,France,FR,5.122075,45.652620,0.010263,43.240029,707.002,648.745035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407386,2023-02-15 14:43:00,FR45,DC Evreux,1500074109,FRANCE BATIMENT INDUSTRIE,34 RUE BOIS GALON,94120,FONTENAY SOUS BOIS,FR,41,DPD FRANCE,0.593,0.009,NaN,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,2.493596,48.859299,112.514,6.672080
407387,2023-02-20 14:34:00,FR45,DC Evreux,1000061780,NOLLET SAS EVREUX,823 RUE JACQUART,27000,EVREUX,FR,41,DPD FRANCE,1.139,0.010,NaN,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,1.136429,49.001823,5.883,0.670074
407388,2022-10-28 08:58:00,FR45,DC Evreux,1500140612,AMAZON FRANCE FC XFRN,7 RUE DES QUATRES ROUTES,38070,ST QUENTIN FALLAVIER,FR,49,CALBERSON ILE DE FRANCE,3.128,0.027,NaN,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,5.122949,45.655422,584.224,182.745267
407389,2022-11-02 10:31:00,FR45,DC Evreux,1500140612,AMAZON FRANCE FC XFRN,7 RUE DES QUATRES ROUTES,38070,ST QUENTIN FALLAVIER,FR,49,CALBERSON ILE DE FRANCE,22.000,0.720,NaN,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,5.122949,45.655422,584.224,1285.292800


In [90]:
df['DC Name'].unique()

array(['RDC Oerebro', 'DC Newlog', 'DC Evreux', 'DC Ceelog'], dtype=object)

In [ ]:
df1['DC Name']

In [77]:
df1.shape

(145364, 27)

In [14]:
print(df.columns.to_list())

['Order number', 'Shipment number', 'Receiver', 'Receiver address', 'Carrier', 'Product', 'Current Status Timestamp', 'Current Status', 'Sender price 1', 'Carrier Price 1', 'Receiver zip', 'Submitted', 'Latest mobile event', 'Actor', 'Sender price 2', 'Fixed price', 'Sender weight (kg)', 'Sender volume (l)', 'Carrier price 2', 'Carrier Weight (kg)', 'Carrier Volume (l)', 'Receiver country code', 'Pickup date', 'Number of packages', 'Subcarrier', 'Goods type', 'Message to receiver', 'Receiver reference', 'Loadmeter', 'Sender', 'Project name', 'Sender account at carrier', 'Payer account at carrier', 'Custom field 1', 'Custom field 2', 'Custom field 3', 'Custom field 4', 'Custom field 5', 'If insurance amount', 'If insurance currency', 'IfInsurancePremium', 'Receiver Freight payer account', 'Receiver Customer No At Sender', 'Signature name', 'First package content', 'Services', 'Operator', 'Search key', 'COD Amount', 'COD Currency', 'Additional Order Number', 'Receiver city', 'Wanted deli

In [18]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report", minimal=True)
profile.to_file(f'output/nshift_oerebro_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\SESA667490\venvs\sustainable-transport\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\SESA667490\venvs\sustainable-transport\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\SESA667490\venvs\sustainable-transport\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\SESA667490\venvs\sustainable-transport\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\SESA667490\venvs\sustainable-transport\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
df.Sender.unique()

array(['Schneider Electric Distributon Center AB',
       'Schneider Electric Distribution Center AB', 'Elko AB',
       'Schneider Electric AB', 'Schneider Electric Oy',
       'Schneider Eletric Norge AS', 'Schneider Electric RDC AB',
       'Elko AS', nan, 'Schneider Electric ', 'Schneider Electric DC AB',
       'Schneider Electric', 'Elko ABS',
       'Schnieder Electric Sverige Distribution Centre AB', '0',
       'Schneider Electric Norge AS', 'Hel Palle'], dtype=object)

In [24]:
dict_column_names = 'nshift_names'

client = boto3.client('location')
s3 = s3fs.S3FileSystem()


logger.info('Preprocessing has started')
# Import new data and dictionary
with s3.open(f'{project_path}/external/{dict_column_names}.json', 'r') as fp:
    dict_names = json.load(fp)

# Rename column names based on given dictionary
df = df.rename({y: x for x, y in dict_names.items()}, axis = 1)

2023-03-07 11:20:26.311 | INFO     | __main__:<module>:7 - Preprocessing has started


In [61]:
df = df.rename({y: x for x, y in dict_names.items()}, axis = 1)

df = df.dropna(subset = ['Pickup date', 'DC name'])

df = df[df['Country code'] != '0']

df = df[dict_names.keys()]

df['DC code'] = 'DK01'
df['Shipping type'] = 'Road'

df = get_country_name_general(df)

In [62]:
df.head()

,Pickup date,Client no,DC name,Client name,Address,Zip,City,Country code,Carrier name,Sender weight (kg),Volume (m3),Nb of packages,DC code,Shipping type,Country
0,14/01/2022 00:00:00,47913,RDC Örebro,AXELENT AB,KÄVSJÖVÄGEN 17 PORT 3 EL 4,33573,HILLERSTORP,SE,Schenker Sverige (SE),7.590,5.4529,1.0,DK01,Road,Sweden
1,14/01/2022 00:00:00,20003801,RDC Örebro,PMC EL,Kolonnvägen 2,16970,NaN,SE,DHL Sverige (SE),445.000,14910.01,5.0,DK01,Road,Sweden
2,14/01/2022 00:00:00,13147,RDC Örebro,BORÅS ELSKÅP AB,SOLVARVSGATAN 4,50740,BORÅS,SE,DHL Sverige (SE),8.804,12.7332,3.0,DK01,Road,Sweden
3,14/01/2022 00:00:00,SESA526477,RDC Örebro,PerPR AB,Stora Nygatan 21B,11127,Stockholm,SE,DHL Sverige (SE),0.640,2.9218,2.0,DK01,Road,Sweden
4,14/01/2022 00:00:00,IN111106,RDC Örebro,EA ELAUTOMATION AB,NAVIGATÖRSGATAN 11,72132,VÄSTERÅS,SE,DHL Sverige (SE),4.510,18.1786,4.0,DK01,Road,Sweden


In [63]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report", minimal=True)
profile.to_file(f'output/nshift_oerebro_after_prep.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [65]:
df['Volume (m3)'].astype('float')

0             5.4529
1         14910.0100
2            12.7332
3             2.9218
4            18.1786
             ...    
133398        5.2234
133399        3.6968
133400       45.7094
133401        0.3400
133402        8.3520
Name: Volume (m3), Length: 133394, dtype: float64

In [30]:
df_small

,Pickup date,Client no,DC name,Client name,Address,Zip,City,Country code,Carrier name,Sender weight (kg),Volume (m3),Nb of packages
0,14/01/2022 00:00:00,47913,RDC Örebro,AXELENT AB,KÄVSJÖVÄGEN 17 PORT 3 EL 4,33573,HILLERSTORP,SE,Schenker Sverige (SE),7.590,5.4529,1.0
1,14/01/2022 00:00:00,20003801,RDC Örebro,PMC EL,Kolonnvägen 2,16970,NaN,SE,DHL Sverige (SE),445.000,14910.01,5.0
2,14/01/2022 00:00:00,13147,RDC Örebro,BORÅS ELSKÅP AB,SOLVARVSGATAN 4,50740,BORÅS,SE,DHL Sverige (SE),8.804,12.7332,3.0
3,14/01/2022 00:00:00,SESA526477,RDC Örebro,PerPR AB,Stora Nygatan 21B,11127,Stockholm,SE,DHL Sverige (SE),0.640,2.9218,2.0
4,14/01/2022 00:00:00,IN111106,RDC Örebro,EA ELAUTOMATION AB,NAVIGATÖRSGATAN 11,72132,VÄSTERÅS,SE,DHL Sverige (SE),4.510,18.1786,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
133398,21/07/2022 00:00:00,4505239011,RDC Örebro,"Wartsila Norway A/S,",Meatjønsvegen 74,5412,STORD,NO,Bring (NO),0.948,5.2234,1
133399,21/07/2022 00:00:00,4802004232,RDC Örebro,Lysteknikk AS,Ulvenveien 92B,581,Oslo,NO,Bring (NO),1.368,3.6968,1
133400,21/07/2022 00:00:00,95940,RDC Örebro,EID ELEKTRO AS,ØYANE 12,6770,NORDFJORDEID,NO,Bring (NO),24.200,45.7094,2
133401,21/07/2022 00:00:00,13644,RDC Örebro,BLUETEC AS,STOLTENBERGS GATE 48,3117,TØNSBERG,NO,Bring (NO),0.239,0.34,1


In [43]:
df_small.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133398 entries, 0 to 133402
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Pickup date         133398 non-null  object 
 1   Client no           132547 non-null  object 
 2   DC name             133395 non-null  object 
 3   Client name         133398 non-null  object 
 4   Address             133370 non-null  object 
 5   Zip                 133391 non-null  object 
 6   City                99233 non-null   object 
 7   Country code        133398 non-null  object 
 8   Carrier name        133398 non-null  object 
 9   Sender weight (kg)  133395 non-null  float64
 10  Volume (m3)         133397 non-null  object 
 11  Nb of packages      133398 non-null  object 
dtypes: float64(1), object(11)
memory usage: 17.3+ MB


In [52]:
df_small = df_small.dropna(subset = ['Pickup date', 'DC name'])

In [54]:
df_small.head()

,Pickup date,Client no,DC name,Client name,Address,Zip,City,Country code,Carrier name,Sender weight (kg),Volume (m3),Nb of packages
0,14/01/2022 00:00:00,47913,RDC Örebro,AXELENT AB,KÄVSJÖVÄGEN 17 PORT 3 EL 4,33573,HILLERSTORP,SE,Schenker Sverige (SE),7.590,5.4529,1.0
1,14/01/2022 00:00:00,20003801,RDC Örebro,PMC EL,Kolonnvägen 2,16970,NaN,SE,DHL Sverige (SE),445.000,14910.01,5.0
2,14/01/2022 00:00:00,13147,RDC Örebro,BORÅS ELSKÅP AB,SOLVARVSGATAN 4,50740,BORÅS,SE,DHL Sverige (SE),8.804,12.7332,3.0
3,14/01/2022 00:00:00,SESA526477,RDC Örebro,PerPR AB,Stora Nygatan 21B,11127,Stockholm,SE,DHL Sverige (SE),0.640,2.9218,2.0
4,14/01/2022 00:00:00,IN111106,RDC Örebro,EA ELAUTOMATION AB,NAVIGATÖRSGATAN 11,72132,VÄSTERÅS,SE,DHL Sverige (SE),4.510,18.1786,4.0


In [55]:
df_small['Client name'].unique()

array(['AXELENT AB', 'PMC EL', 'BORÅS ELSKÅP AB', ...,
       'BT EL PER BLOM AB', 'Morakniv', 'OY KONTAKTOR AB'], dtype=object)

In [53]:
df_small[df_small['DC name'].isna()]

,Pickup date,Client no,DC name,Client name,Address,Zip,City,Country code,Carrier name,Sender weight (kg),Volume (m3),Nb of packages
57028,FR,NaN,NaN,5990907676,Cargotec Sweden AB C/O NEOVIA LOGISTICS SERVICES,NaN,NaN,0,Batiment 2,NaN,7.8,30/08/2022 00:00:00
79611,FO,NaN,NaN,8263462491,El-service P/F,NaN,NaN,17.732,Bryggjan 15,NaN,6.1,20/04/2022 00:00:00
104748,DK,NaN,NaN,DU737810,Schneider Electric,307.60,NaN,0,orbaekvej 280,NaN,32.0,26/07/2022 00:00:00
